In [1]:
using Gridap
using GridapGmsh

In [7]:
model = GmshDiscreteModel("cascara_10_4.msh")
#model = GmshDiscreteModel("cascara_10_2.msh")
#fn = "model.json"
#to_json_file(model,fn)

Info    : Reading 'cascara_10_4.msh'...
Info    : 13 entities
Info    : 7161 nodes
Info    : 39355 elements
Info    : Done reading 'cascara_10_4.msh'


UnstructuredDiscreteModel()

In [8]:
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
#V = TestFESpace(model,reffe,dirichlet_tags=["int","ext"])
V = TestFESpace(model,reffe;conformity=:H1,dirichlet_tags="int")
Ω = Triangulation(model)

UnstructuredGrid()

In [9]:
writevtk(Ω,"sphere")

(["sphere.vtu"],)

In [10]:
g = 2.0 # internal boundary condition
U = TrialFESpace(V,g)

TrialFESpace()

In [11]:
degree = 2
dΩ = Measure(Ω,degree)

Measure()

In [12]:
neumanntags = "ext"
Γ = BoundaryTriangulation(model,tags=neumanntags)
dΓ = Measure(Γ,degree)

Measure()

### Test 1: ###

We first test with given boundary values: $\phi = 2$ at $R_{min}$ and $\nabla \phi = \frac{1}{R_{max}}$ at $R_{max}$. The solution is: 

$$
\phi(r) = \frac{a}{r} + b \;\;\;\;\;\;\; \hat{n} \cdot \nabla \phi = -\frac{a}{r^2}
$$ 

From which, $a = - R_{max}$ and $b = 2 + \frac{R_{max}}{R_{min}}$

This gives, $\phi_{R_{max}} = 4.5$. And it is OK

### Test 2: ###

We now impose Robin BC. 

$$
\hat{n} \cdot \nabla \phi = -\frac{\phi}{r} \;\;\;\; \mbox{at} \;\; r = R_{max}
$$

For this case, $\phi = \frac{a}{r}$ and so, $\phi(R_{min} = 2$ implies $a = 2R_{min}$, 
and so $ \phi(R_{max}) = \frac{2R_{min}}{R_{max}}$.

Test2 passed! OK





In [31]:
test1=false
test2=true

if test1
    R_ext = 10.
    R_int = 4.
    f(x) = 0.0 # source
    h(x) = 1.0/R_ext #external Neumann bc.
    a(u,v) = ∫( ∇(v)⋅∇(u) )*dΩ
    b(v) = ∫( v*f )*dΩ + ∫( v*h )*dΓ
end

if test2
    R_ext = 10.
    R_int = 4.
    f(x) = 0.0 # source
    h(x) = -1.0/R_ext #external Neumann bc.
    a(u,v) = ∫( ∇(v)⋅∇(u) )*dΩ - ∫( v*h*u )*dΓ
    b(v) = ∫( v*f )*dΩ 
end

b (generic function with 1 method)

In [32]:
op = AffineFEOperator(a,b,U,V)


AffineFEOperator()

In [33]:
ls = LUSolver()
solver = LinearFESolver(ls)


LinearFESolver()

In [34]:
uh = solve(solver,op)

SingleFieldFEFunction():
 num_cells: 33179
 DomainStyle: ReferenceDomain()
 Triangulation: UnstructuredGrid()
 Triangulation id: 2631304195112948753

In [35]:
writevtk(Ω,"results_test2",cellfields=["uh"=>uh])

(["results_test2.vtu"],)